In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multilingual-customer-support-tickets/dataset-tickets-multi-lang-4-20k.csv
/kaggle/input/multilingual-customer-support-tickets/dataset-tickets-german_normalized_50_5_2.csv
/kaggle/input/multilingual-customer-support-tickets/dataset-tickets-multi-lang3-4k.csv
/kaggle/input/multilingual-customer-support-tickets/dataset-tickets-german_normalized.csv
/kaggle/input/multilingual-customer-support-tickets/aa_dataset-tickets-multi-lang-5-2-50-version.csv


In [2]:
!pip -q install accelerate trl bitsandbytes rouge_score bert_score peft evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00


In [3]:
import os
import math
import random
import pandas as pd
import numpy as np

import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer
import evaluate

2026-01-19 05:42:08.353477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768801328.554174      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768801328.609865      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768801329.230835      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768801329.230876      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768801329.230879      55 computation_placer.cc:177] computation placer alr

In [4]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
if device == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

Device: cuda
GPU: Tesla P100-PCIE-16GB


In [5]:
DATA_PATH = "/kaggle/input/multilingual-customer-support-tickets/aa_dataset-tickets-multi-lang-5-2-50-version.csv"
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
OUTPUT_DIR = "/kaggle/working/qlora_ticket_support_model"

MAX_LENGTH = 1024
TRAIN_RATIO = 0.96
VAL_RATIO = 0.02
TEST_RATIO = 0.02

GEN_MAX_NEW_TOKENS = 180
GEN_TEMPERATURE = 0.0
GEN_DO_SAMPLE = False

EVAL_SAMPLE_SIZE = 300

In [6]:
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head(3)

Shape: (28587, 16)
Columns: ['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language', 'version', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8']


,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,Outage,Disruption,Data Breach,NaN,NaN,NaN,NaN
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,Disruption,Outage,IT,Tech Support,NaN,NaN,NaN
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,Feature,Tech Support,NaN,NaN,NaN,NaN,NaN


In [7]:
SYSTEM_PROMPT = (
    "You are a helpful, professional customer support agent. "
    "Write a clear, polite and actionable reply. "
    "If information is missing, ask concise follow-up questions. "
    "Keep the same language as the user's ticket.")

def make_example(row):
    subject = str(row["subject"]) if pd.notna(row["subject"]) else ""
    body = str(row["body"]) if pd.notna(row["body"]) else ""
    answer = str(row["answer"]) if pd.notna(row["answer"]) else ""

    user_msg = (
        "Please draft a customer support reply.\n\n"
        f"Subject: {subject}\n\n"
        f"Body:\n{body}\n")

    prompt = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_msg},]
    completion = [
        {"role": "assistant", "content": answer}]
    return {
        "prompt": prompt,
        "completion": completion,
        "language": row.get("language", None),
        "priority": row.get("priority", None),
        "queue": row.get("queue", None),
        "type": row.get("type", None)}

examples = [make_example(r) for _, r in df.iterrows()]
ds = Dataset.from_list(examples)
ds

Dataset({
    features: ['prompt', 'completion', 'language', 'priority', 'queue', 'type'],
    num_rows: 28587
})

In [8]:
tmp = ds.train_test_split(test_size=(1.0 - TRAIN_RATIO), seed=SEED, shuffle=True)
train_ds = tmp["train"]
temp_ds = tmp["test"]
val_size = VAL_RATIO / (VAL_RATIO + TEST_RATIO)
tmp2 = temp_ds.train_test_split(test_size=(1.0 - val_size), seed=SEED, shuffle=True)
val_ds = tmp2["train"]
test_ds = tmp2["test"]

print("Train:", len(train_ds))
print("Val:  ", len(val_ds))
print("Test: ", len(test_ds))

train_ds[0]

Train: 27443
Val:   572
Test:  572


{'prompt': [{'content': "You are a helpful, professional customer support agent. Write a clear, polite and actionable reply. If information is missing, ask concise follow-up questions. Keep the same language as the user's ticket.",
   'role': 'system'},
  {'content': 'Please draft a customer support reply.\n\nSubject: \n\nBody:\nHello Customer Support, I am inquiring about optimizing investment strategies using data analytics products and services. Could you provide some insights on how to leverage these tools to make informed decisions? I would greatly appreciate any guidance on best practices for implementing data analytics in investment strategies. Additionally, could you let me know what products and services are available to assist in this process? Thank you for your time and assistance. I look forward to hearing back soon.\n',
   'role': 'user'}],
 'completion': [{'content': 'Dear [Name], we appreciate your inquiry regarding optimizing investment strategies using data analytics p

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True)
model.config.use_cache = False

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [10]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"])

training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    max_length=MAX_LENGTH,
    packing=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    bf16=True,
    gradient_checkpointing=True,
    report_to="none")

In [11]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    peft_config=peft_config)

trainer.train()

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

Tokenizing train dataset:   0%|          | 0/27443 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/27443 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/572 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/572 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss
200,1.074400,1.100415


('/kaggle/working/qlora_ticket_support_model/tokenizer_config.json',
 '/kaggle/working/qlora_ticket_support_model/special_tokens_map.json',
 '/kaggle/working/qlora_ticket_support_model/chat_template.jinja',
 '/kaggle/working/qlora_ticket_support_model/vocab.json',
 '/kaggle/working/qlora_ticket_support_model/merges.txt',
 '/kaggle/working/qlora_ticket_support_model/added_tokens.json',
 '/kaggle/working/qlora_ticket_support_model/tokenizer.json')

In [12]:
eval_out = trainer.evaluate()
print(eval_out)

if "eval_loss" in eval_out:
    ppl = math.exp(eval_out["eval_loss"]) if eval_out["eval_loss"] < 20 else float("inf")
    print("Validation perplexity:", ppl)

{'eval_loss': 1.075737476348877, 'eval_runtime': 113.9078, 'eval_samples_per_second': 1.194, 'eval_steps_per_second': 0.597}
Validation perplexity: 2.9321544979657443


In [13]:
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

n_test = len(test_ds)
n_eval = min(EVAL_SAMPLE_SIZE, n_test)
test_subset = test_ds.select(range(n_eval))

print("Evaluating on:", n_eval, "samples")

def generate_answer_from_prompt(prompt_messages):
    input_ids = tokenizer.apply_chat_template(
        prompt_messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt")

    input_ids = input_ids.to(model.device)
    with torch.no_grad():
        gen_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=GEN_MAX_NEW_TOKENS,
            do_sample=GEN_DO_SAMPLE,
            temperature=GEN_TEMPERATURE,
            top_p=1.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id)
    gen_new = gen_ids[0][input_ids.shape[-1]:]
    text = tokenizer.decode(gen_new, skip_special_tokens=True).strip()
    return text

Evaluating on: 300 samples


In [14]:
preds = []
refs = []
for i in range(n_eval):
    ex = test_subset[i]
    prompt = ex["prompt"]
    ref = ex["completion"][0]["content"]
    pred = generate_answer_from_prompt(prompt)
    preds.append(pred)
    refs.append(ref)
    if (i + 1) % 25 == 0:
        print("Done", i + 1, "/", n_eval)
print("Sample prediction:\n", preds[0])
print("\nReference:\n", refs[0])

rouge_scores = rouge.compute(predictions=preds, references=refs)
bert_scores = bertscore.compute(
    predictions=preds,
    references=refs,
    model_type="bert-base-multilingual-cased",
    rescale_with_baseline=False)

print("ROUGE:", {k: round(v, 4) for k, v in rouge_scores.items()})
print("BERTScore (F1 avg):", round(float(np.mean(bert_scores["f1"])), 4))

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Done 25 / 300
Done 50 / 300
Done 75 / 300
Done 100 / 300
Done 125 / 300
Done 150 / 300
Done 175 / 300
Done 200 / 300
Done 225 / 300
Done 250 / 300
Done 275 / 300
Done 300 / 300
Sample prediction:
 Sehr geehrte [Name], vielen Dank für Ihre Nachricht bezüglich des Verbindungsproblems mit Ihrem Google Nest Wifi Router. Ich verstehe, dass dies die Nutzung von Online-Bereitstellungen und der Kommunikation mit Ihren Kunden beeinträchtigt hat. Um Ihnen besser zu helfen, bitte ich Sie, mehr Details über die neuesten Firmware-Updates und die Netzwerkkonfiguration zu geben. Falls notwendig, können wir einen Anruf vereinbaren, um das Problem genauer zu besprechen. Bitte teilen Sie mir eine passende Zeit mit, damit wir Kontakt aufnehmen können.

Reference:
 Wir sind Ihnen bei den Verbindungsproblemen mit Ihrem Google Nest Wifi Router behilflich. Für ein präzises Troubleshooting könnten Sie bitte den Modellnamen und die Firmware-Version des Routers nennen. Eine gute Zeit für einen Anruf ist <tel_nu

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

ROUGE: {'rouge1': np.float64(0.3837), 'rouge2': np.float64(0.1553), 'rougeL': np.float64(0.2652), 'rougeLsum': np.float64(0.2647)}
BERTScore (F1 avg): 0.7551


In [17]:
demo_subject = "Refund request for duplicate charge"

demo_body = (
    "Hi, I was charged twice for order #A18473 on January 12. "
    "I only placed one order and the second payment is pending in my bank. "
    "Can you confirm if this is a duplicate charge and refund it? "
    "Attached: screenshot of the two transactions.")

demo_prompt = [{"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Please draft a customer support reply.\n\nSubject: "
     + demo_subject + "\n\nBody:\n" + demo_body + "\n"}]

demo_answer = generate_answer_from_prompt(demo_prompt)
print(demo_answer)

We will investigate the duplicate charge issue with order A18473. Please provide your account number so we can proceed with the refund process.
